# Gesture Recognition
## Ram Krishn Mishra
## Sairaz Kota

In [1]:
#import the following libraries to get started.

import os
import numpy as np
from scipy.misc import imread, imresize
import cv2
import datetime

We need to setup the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)

import random as rn
rn.seed(30)

In [3]:
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

Using TensorFlow backend.


Now, we will read the folder names for training and validation. We also set the `batch_size` here. Note that we need to set the batch size in such a way that we are able to use the GPU in full capacity. we keep increasing the batch size until the machine throws an error.

In [4]:
train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())
batch_size = 10 #experiment with the batch size

### Generator

This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, we are going to preprocess the images as we have images of 2 different dimensions as well as create a batch of video frames. we have to experiment with `img_idx`, `y`,`z` and normalization such that we get high accuracy.

In [5]:
#Initialize the params

nb_rows = 120   # X dimension of the image
nb_cols = 120   # Y dimesnion of the image

#total_frames = 30

nb_frames = 30  # length of the video frames
nb_channel = 3 # number of channels in images 3 for RGB color & 1 for Grey

In [6]:
# Util function to initialize all the batch image labels and respective data

def init_batch_data(batch_size):
    batch_data = np.zeros((batch_size, nb_frames, nb_rows, nb_cols, nb_channel)) 
    batch_labels = np.zeros((batch_size,5)) # batch_labels is the one best representation of the output
    return batch_data, batch_labels

In [7]:
# Util function to normalise the data

def normalize_data(data):
    return data/127.5-1

In [8]:
# Util function to create/ generate a random affine transformation on the image

def get_random_affine():
    dx, dy = np.random.randint(-1.7, 1.8, 2)
    M = np.float32([[1, 0, dx], [0, 1, dy]])
    return M

In [9]:
def load_batch_images(source_path, folder_list, batch_num, batch_size, t, validation):
    
    batch_data, batch_labels = init_batch_data(batch_size)
    
    # We will also build an augumented batch data with affine transformation
    batch_data_aug, batch_labels_aug = init_batch_data(batch_size)
    
    # We will also build an augmented batch data with horizontal flip
    batch_data_flip, batch_labels_flip = init_batch_data(batch_size)
    
    # Lets create a list of image numbers you want to use for a particular video using full frames
    img_idx = [x for x in range(0, nb_frames)]
    

    for folder in range(batch_size):
        # Iterate all the images in the folder with the batch size
        
        imgs = sorted(os.listdir(source_path+'/'+ t[folder + (batch_num * batch_size)].split(';')[0]))
        
        # Generate a random affine transformation to be used in image transformation for -
        #buidling agumented data set
        
        M = get_random_affine()
        
        
        
        for idx, item in enumerate(img_idx): 
            #  Iterate over the frames/images of a folder to read them in
            image = cv2.imread(source_path+'/'+ t[folder + (batch_num*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
            
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # Lets crop non symmetric frames
            if image.shape[0] != image.shape[1]:
                image = image[0:120,20:140]
            
            # Crop the images and resize them as the images are of 2 different shape & the conv3D will throw error 
            # If the inputs are in a batch with different shapes   
            resized = cv2.resize(image, (nb_rows,nb_cols), interpolation = cv2.INTER_AREA)
            
            # Normalize data
            batch_data[folder, idx] = (resized)
            
            # Data with affine transformation
            batch_data_aug[folder, idx] = (cv2.warpAffine(resized, M, (resized.shape[0], resized.shape[1])))
            
            # Data with horizontal flip
            batch_data_flip[folder, idx]= np.flip(resized, 1)

        batch_labels[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
        batch_labels_aug[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
        
        # Labeling data with horizobtal flip, right swipe becomes left swipe & viceversa
        if int(t[folder + (batch_num*batch_size)].strip().split(';')[2])==0:
                    batch_labels_flip[folder, 1] = 1
        elif int(t[folder + (batch_num*batch_size)].strip().split(';')[2])==1:
                    batch_labels_flip[folder, 0] = 1
                    
        else:
                    batch_labels_flip[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
                  
    
    batch_data_final = np.append(batch_data, batch_data_aug, axis = 0)
    batch_data_final = np.append(batch_data_final, batch_data_flip, axis = 0)

    batch_labels_final = np.append(batch_labels, batch_labels_aug, axis = 0) 
    batch_labels_final = np.append(batch_labels_final, batch_labels_flip, axis = 0)
    
    if validation:
        batch_data_final=batch_data
        batch_labels_final= batch_labels
        
    return batch_data_final,batch_labels_final

In [10]:
def generator(source_path, folder_list, batch_size, validation=False):
    
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        
        for batch in range(num_batches):        # we iterate over the number of batches
            
            # we yield the batch_data and the batch_labels, remember what does yield do
            yield load_batch_images(source_path, folder_list, batch, batch_size, t,validation)
            

        
        # For the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            
            batch_size = len(folder_list) - (batch_size*num_batches)
            yield load_batch_images(source_path, folder_list, batch, batch_size, t,validation)


the generator as (number of images, height, width, number of channels). we need take this into consideration while creating the model architecture.

In [11]:
curr_dt_time = datetime.datetime.now()

train_path = './Project_data/train'

val_path = './Project_data/val'


In [12]:
num_train_sequences = len(train_doc)
print('No of training sequences =', num_train_sequences)

No of training sequences = 663


In [13]:
num_val_sequences = len(val_doc)
print('No of validation sequences =', num_val_sequences)

No of validation sequences = 100


In [14]:
num_epochs = 10 # the number of epochs
num_epochs

10

## Model
Here we make the model using different functionalities that Keras provides. We have to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. we would want to use `TimeDistributed` while building a Conv2D + RNN model. Also we need that the last layer is the softmax. we need to design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [15]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam

In [16]:
nb_filters = [8,16,32,64]
nb_dense = [256, 128, 5]

# Input
input_shape=(30,nb_rows,nb_cols,3)

# Define model
model = Sequential()


model.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2)))


model.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2)))



model.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2)))



model.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(Activation('relu'))

model.add(Dropout(0.25))

model.add(MaxPooling3D(pool_size=(2,2,2)))

In [17]:
#Flatten Layers

model.add(Flatten())
model.add(Dense(nb_dense[0], activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_dense[1], activation='relu'))
model.add(Dropout(0.5))

In [18]:
#softmax layer

model.add(Dense(nb_dense[2], activation='softmax'))

Now that we have written the model, the next step is to `compile` the model. When we print the `summary` of the model, we'll see the total number of parameters that we have to train.

In [19]:
optimiser = Adam() #optimizer to avoid the perfomance issue
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

print (model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_1 (Activation)    (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_2 (Batch (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_2 (Activation)    (None, 15, 60, 60, 16)   

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [20]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [21]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [22]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
    
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
    
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, we'll be able to save the model at the end of each epoch.

In [23]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/10
Source path =  ./Project_data/val ; batch size = 10
Source path =  ./Project_data/train ; batch size = 10
67/67 [==============================] - 282s 4s/step - loss: 1.6768 - categorical_accuracy: 0.2916 - val_loss: 2.6635 - val_categorical_accuracy: 0.3333

Epoch 00001: saving model to model_init_2020-05-0415_24_46.301761/model-00001-1.68391-0.29160-2.66349-0.33333.h5
Epoch 2/10
67/67 [==============================] - 117s 2s/step - loss: 1.3827 - categorical_accuracy: 0.3947 - val_loss: 1.4352 - val_categorical_accuracy: 0.4967

Epoch 00002: saving model to model_init_2020-05-0415_24_46.301761/model-00002-1.38271-0.39469-1.43517-0.49667.h5
Epoch 3/10
67/67 [==============================] - 116s 2s/step - loss: 1.3589 - categorical_accuracy: 0.4693 - val_loss: 1.3318 - val_categorical_accuracy: 0.4833

Epoch 00003: saving model to model_init_2020-05-0415_24_46.301761/model-00003-1.35887-0.46932-1.33183-0.48333.h5
Epoch 4/10
67/67 [==============================] - 116s 

In [25]:
batch_size = 20
num_epochs = 20

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20
34/34 [==============================] - 61s 2s/step - loss: 0.8669 - categorical_accuracy: 0.6438 - val_loss: 0.7043 - val_categorical_accuracy: 0.7667

Epoch 00001: saving model to model_init_2020-05-0415_24_46.301761/model-00001-0.86686-0.64379-0.70434-0.76667.h5
Epoch 2/20
34/34 [==============================] - 58s 2s/step - loss: 0.7513 - categorical_accuracy: 0.7190 - val_loss: 0.9429 - val_categorical_accuracy: 0.5733

Epoch 00002: saving model to model_init_2020-05-0415_24_46.301761/model-00002-0.75130-0.71895-0.94286-0.57333.h5
Epoch 3/20
34/34 [==============================] - 58s 2s/step - loss: 0.6723 - categorical_accuracy: 0.7059 - val_loss: 0.8812 - val_categorical_accuracy: 0.7133

Epoch 00003: saving model to model_init_2020-05-0415_24_46.301761/model-00003-0.67228-0.70588-0.88124-0.71333.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
Epoch 4/20
34/34 [==============================] - 58s 2s/step - loss: 0.7773 - cate

In [28]:
batch_size = 30
num_epochs = 30

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
23/23 [==============================] - 42s 2s/step - loss: 0.4974 - categorical_accuracy: 0.7971 - val_loss: 0.4490 - val_categorical_accuracy: 0.7417

Epoch 00001: saving model to model_init_2020-05-0415_24_46.301761/model-00001-0.49745-0.79710-0.44897-0.74167.h5
Epoch 2/30
23/23 [==============================] - 41s 2s/step - loss: 0.4412 - categorical_accuracy: 0.8261 - val_loss: 0.8202 - val_categorical_accuracy: 0.7500

Epoch 00002: saving model to model_init_2020-05-0415_24_46.301761/model-00002-0.44120-0.82609-0.82019-0.75000.h5
Epoch 3/30
23/23 [==============================] - 41s 2s/step - loss: 0.5114 - categorical_accuracy: 0.7729 - val_loss: 0.6415 - val_categorical_accuracy: 0.7500

Epoch 00003: saving model to model_init_2020-05-0415_24_46.301761/model-00003-0.51137-0.77295-0.64149-0.75000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 3.814697446813398e-09.
Epoch 4/30
23/23 [==============================] - 41s 2s/step - loss: 0.4935 - cate

In [26]:
batch_size = 50
num_epochs = 30

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
14/14 [==============================] - 25s 2s/step - loss: 0.3612 - categorical_accuracy: 0.9048 - val_loss: 0.5561 - val_categorical_accuracy: 0.7333

Epoch 00001: saving model to model_init_2020-05-0415_24_46.301761/model-00001-0.36116-0.90476-0.55613-0.73333.h5
Epoch 2/30
14/14 [==============================] - 24s 2s/step - loss: 0.5587 - categorical_accuracy: 0.8095 - val_loss: 0.7101 - val_categorical_accuracy: 0.7500

Epoch 00002: saving model to model_init_2020-05-0415_24_46.301761/model-00002-0.55866-0.80952-0.71011-0.75000.h5
Epoch 3/30
14/14 [==============================] - 25s 2s/step - loss: 0.4992 - categorical_accuracy: 0.8175 - val_loss: 0.7491 - val_categorical_accuracy: 0.7667

Epoch 00003: saving model to model_init_2020-05-0415_24_46.301761/model-00003-0.49916-0.81746-0.74914-0.76667.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 1.5625000742147677e-05.
Epoch 4/30
14/14 [==============================] - 25s 2s/step - loss: 0.5006 - cat

## Model CNN-RNN

In [30]:
# Following is the model constructed for building a Conv2D + RNN model. 
from keras.layers import Conv2D, MaxPooling2D, LSTM

model = Sequential()

model.add(TimeDistributed(Conv2D(32, (7, 7), strides=(2, 2),
            activation='relu', padding='same'), input_shape=input_shape))
model.add(TimeDistributed(Conv2D(32, (3,3),
            kernel_initializer="he_normal", activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Conv2D(64, (3,3),
            padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(64, (3,3),
            padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Conv2D(128, (3,3),
            padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(128, (3,3),
            padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Conv2D(256, (3,3),
            padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(256, (3,3),
            padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
        
model.add(TimeDistributed(Conv2D(512, (3,3),
            padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(512, (3,3),
            padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Flatten()))

model.add(Dropout(0.5))
model.add(LSTM(256, return_sequences=False, dropout=0.5))
model.add(Dense(5, activation='softmax'))

In [31]:
optimiser = Adam() #optimizer to avoid the perfomance issue
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_17 (TimeDis (None, 30, 60, 60, 32)    4736      
_________________________________________________________________
time_distributed_18 (TimeDis (None, 30, 58, 58, 32)    9248      
_________________________________________________________________
time_distributed_19 (TimeDis (None, 30, 29, 29, 32)    0         
_________________________________________________________________
time_distributed_20 (TimeDis (None, 30, 29, 29, 64)    18496     
_________________________________________________________________
time_distributed_21 (TimeDis (None, 30, 29, 29, 64)    36928     
_________________________________________________________________
time_distributed_22 (TimeDis (None, 30, 14, 14, 64)    0         
_________________________________________________________________
time_distributed_23 (TimeDis (None, 30, 14, 14, 128)  

In [32]:
batch_size = 20
num_epochs = 20

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20
34/34 [==============================] - 47s 1s/step - loss: 1.9144 - categorical_accuracy: 0.2353 - val_loss: 1.7488 - val_categorical_accuracy: 0.2133

Epoch 00001: saving model to model_init_2020-05-0415_24_46.301761/model-00001-1.91440-0.23529-1.74876-0.21333.h5
Epoch 2/20
34/34 [==============================] - 40s 1s/step - loss: 1.8318 - categorical_accuracy: 0.1765 - val_loss: 1.8046 - val_categorical_accuracy: 0.2200

Epoch 00002: saving model to model_init_2020-05-0415_24_46.301761/model-00002-1.83180-0.17647-1.80460-0.22000.h5
Epoch 3/20
34/34 [==============================] - 40s 1s/step - loss: 1.7560 - categorical_accuracy: 0.1961 - val_loss: 1.4921 - val_categorical_accuracy: 0.2600

Epoch 00003: saving model to model_init_2020-05-0415_24_46.301761/model-00003-1.75597-0.19608-1.49209-0.26000.h5
Epoch 4/20
34/34 [==============================] - 42s 1s/step - loss: 1.7546 - categorical_accuracy: 0.1797 - val_loss: 1.7678 - val_categorical_accuracy: 0.2467

E

Final Result for CNN-RNN is 20% accuracy which is not good for  this data. 
Conv3D is giving best result with accuracy of 82.5%